# Spark Apache (семинары)
## Урок 1. SQL & BigData
Найти самую длинную последовательность упорядоченных чисел в RDD и вывести её.

In [ ]:
from pyspark import SparkContext
import random

In [ ]:
# Функция для нахождения последовательностей по возрастанию
def create_sequences(numbers):
    data = list(numbers)
    if not data:
        return
    
    sequences = []
    current_sequence = [data[0]]
    
    for i in range(1, len(data)):
        if data[i] >= data[i-1]:
            current_sequence.append(data[i])
        else:
            sequences.append(current_sequence)
            current_sequence = [data[i]]
    
    sequences.append(current_sequence)
    
    for seq in sequences:
        yield seq

In [ ]:
# Инициализация Spark сессии
sc = SparkContext("local", "Longest Sequence")

# Создаем RDD датасет из 100 случайных элементов
random_numbers = [random.randint(0, 20) for _ in range(30)]
numbers_rdd = sc.parallelize(random_numbers, 1)  # 1 партиция для одного узла
print("Исходный датасет:", *numbers_rdd.collect(), "\n")

# Рзбиваем датасет на возрастающие последовательности
sequences_rdd = numbers_rdd.mapPartitions(create_sequences)
# print(*sequences_rdd.collect())

longest_sequence = sequences_rdd.reduce(lambda x, y: x if len(x) > len(y) else y)
print("Самая длинная упорядоченная последовательность:",*longest_sequence, "\n")

# Находим самую длинную последовательность
len_sequence = sequences_rdd.map(lambda x: len(x))               #
max_len_sequence = len_sequence.max()
print("Длина последовательности: ", max_len_sequence)

Исходный датасет: 20 1 6 10 11 0 14 15 0 1 19 15 17 18 3 5 12 16 17 2 0 2 5 11 2 11 12 6 12 19 

Самая длинная упорядоченная последовательность: 3 5 12 16 17 

Длина последовательности:  5


In [ ]:
sc.stop()